In [6]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [7]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [8]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

In [9]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView("home_sales")

In [10]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
SELECT date_built AS Year, ROUND(AVG(price),2) AS Average_Price
FROM home_sales
WHERE bedrooms=4
GROUP BY date_built
ORDER BY date_built
""").show()

+----+-------------+
|Year|Average_Price|
+----+-------------+
|2010|    296800.75|
|2011|     302141.9|
|2012|    298233.42|
|2013|    299999.39|
|2014|    299073.89|
|2015|    307908.86|
|2016|    296050.24|
|2017|    296576.69|
+----+-------------+



In [11]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
SELECT date_built AS Year, ROUND(AVG(price),2) AS Average_Price
FROM home_sales
WHERE bedrooms=3 AND bathrooms=3
GROUP BY date_built
ORDER BY date_built
""").show()

+----+-------------+
|Year|Average_Price|
+----+-------------+
|2010|    292859.62|
|2011|    291117.47|
|2012|    293683.19|
|2013|    295962.27|
|2014|    290852.27|
|2015|     288770.3|
|2016|    290555.07|
|2017|    292676.79|
+----+-------------+



In [12]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""
SELECT date_built AS Year, ROUND(AVG(price),2) AS Average_Price
FROM home_sales
WHERE bedrooms=3 AND bathrooms=3 AND floors=2 AND sqft_living>=2000
GROUP BY date_built
ORDER BY date_built DESC
""").show()

+----+-------------+
|Year|Average_Price|
+----+-------------+
|2017|    280317.58|
|2016|     293965.1|
|2015|    297609.97|
|2014|    298264.72|
|2013|    303676.79|
|2012|    307539.97|
|2011|    276553.81|
|2010|    285010.22|
+----+-------------+



In [13]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
SELECT view AS View_Rating, ROUND(AVG(price),2) AS Average_Price
FROM home_sales
WHERE price>=350000
GROUP BY View_Rating
ORDER BY Average_Price DESC
""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+-----------+-------------+
|View_Rating|Average_Price|
+-----------+-------------+
|         91|   1137372.73|
|         97|   1129040.15|
|         84|   1117233.13|
|         75|   1114042.94|
|         89|   1107839.15|
|         78|   1080649.37|
|         77|   1076205.56|
|         87|    1072285.2|
|         86|   1070444.25|
|         82|    1063498.0|
|         90|   1062654.16|
|         99|   1061201.42|
|         76|   1058802.78|
|         85|   1056336.74|
|         95|    1054325.6|
|         98|   1053739.33|
|         81|   1053472.79|
|         83|   1033965.93|
|         94|    1033536.2|
|         88|   1031719.35|
+-----------+-------------+
only showing top 20 rows

--- 0.9687561988830566 seconds ---


In [14]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [15]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [16]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""
SELECT view AS View_Rating, ROUND(AVG(price),2) AS Average_Price
FROM home_sales
WHERE price>=350000
GROUP BY View_Rating
ORDER BY Average_Price DESC
""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+-----------+-------------+
|View_Rating|Average_Price|
+-----------+-------------+
|         91|   1137372.73|
|         97|   1129040.15|
|         84|   1117233.13|
|         75|   1114042.94|
|         89|   1107839.15|
|         78|   1080649.37|
|         77|   1076205.56|
|         87|    1072285.2|
|         86|   1070444.25|
|         82|    1063498.0|
|         90|   1062654.16|
|         99|   1061201.42|
|         76|   1058802.78|
|         85|   1056336.74|
|         95|    1054325.6|
|         98|   1053739.33|
|         81|   1053472.79|
|         83|   1033965.93|
|         94|    1033536.2|
|         88|   1031719.35|
+-----------+-------------+
only showing top 20 rows

--- 0.483165979385376 seconds ---


In [17]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.parquet("parquet_home_sales", partitionBy="date_built", mode="overwrite")

In [18]:
# 11. Read the formatted parquet data.
home_sales_parquet_df = spark.read.parquet("parquet_home_sales")

In [19]:
# 12. Create a temporary table for the parquet data.
home_sales_parquet_df.createOrReplaceTempView("p_home_sales")

In [20]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""
SELECT view AS View_Rating, ROUND(AVG(price),2) AS Average_Price
FROM home_sales
WHERE price>=350000
GROUP BY View_Rating
ORDER BY Average_Price DESC
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----------+-------------+
|View_Rating|Average_Price|
+-----------+-------------+
|         91|   1137372.73|
|         97|   1129040.15|
|         84|   1117233.13|
|         75|   1114042.94|
|         89|   1107839.15|
|         78|   1080649.37|
|         77|   1076205.56|
|         87|    1072285.2|
|         86|   1070444.25|
|         82|    1063498.0|
|         90|   1062654.16|
|         99|   1061201.42|
|         76|   1058802.78|
|         85|   1056336.74|
|         95|    1054325.6|
|         98|   1053739.33|
|         81|   1053472.79|
|         83|   1033965.93|
|         94|    1033536.2|
|         88|   1031719.35|
+-----------+-------------+
only showing top 20 rows

--- 0.5027029514312744 seconds ---


In [21]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [22]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")

False